In [2]:
from bs4 import BeautifulSoup as bs
import requests as req
import json
from pprint import pprint
from time import sleep
from random import uniform
import sqlalchemy as db
from sqlalchemy import exc

DB_HOST = '127.0.0.1'
DB_PORT = '5432'
DB_NAME = 'jptrvl'
DB_USER = 'postgres'
DB_PASS = 'pttjp118'

# conn_string = f'host={DB_HOST} user={DB_USER} dbname={DB_NAME} password={DB_PASS} port={DB_PORT}'

my_headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'
}

listData = []

# url = 'https://www.ptt.cc/bbs/Japan_Travel/index'+str(7296-i)+'.html'

domainName = 'https://www.ptt.cc/bbs/Japan_Travel/index'
pttDomain = 'https://www.ptt.cc'

# total pages = 7296

start_page = 6994
end_page = 6995

sleep_time = uniform(0.1, 0.5)

def getMainData():
    for page in range(start_page, end_page):
        res = req.get(url = domainName + str(page) + '.html', headers = my_headers)
        #print(res.text)
        soup = bs(res.text, "lxml")
        posts = soup.select('div.r-ent')
        for post in posts:
            if post:
                strTitle = post.select_one('div.title > a').get_text()
                strLink = post.select_one('div.title > a').get('href')
                strAuthor = post.select_one('div.meta > div.author').get_text()
                strPushCount = post.select_one('div.nrec').get_text()

                listData.append({
                    "title": strTitle,
                    "link": strLink,
                    "author": strAuthor,
                    "push_count": strPushCount
                })
            else:
                continue
        #print(listData)

def getDetailData():
    for index, articleDict in enumerate(listData):
        res = req.get(url = pttDomain + articleDict['link'], headers = my_headers)
        soup = bs(res.text, "lxml")
        metalines = soup.select('div.article-metaline')
        if len(metalines) == 3:
            strTime = metalines[2].select_one('span.article-meta-value').get_text()
            all_text = soup.select_one('div#main-content').get_text()
            pre_text = all_text.split('--')[0]
            texts = pre_text.split('\n')
            contents = texts[4:]
            content = ''.join(contents)
            listData[index]['publish_time'] = strTime
            listData[index]['content'] = content

            sleep(sleep_time)

        else:
            listData[index]['publish_time'] = None
            listData[index]['content'] = None

    pprint(listData)

def saveJson():
    with open("pttjptrvl_bs.json", "w", encoding="utf-8") as fp:
        fp.write( json.dumps( listData, ensure_ascii=False, indent=2 ) )

def saveMysql():
    engine = db.create_engine(f'mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}')
    conn  = engine.connect()
    print("Connect sucessfully!")
    try:
        cursor = conn.cursor()
        columns = listData[0].keys()
        stat = "INSERT INTO public.jptrvl_post({}) VALUES %s".format(','.join(columns))
        values = [[value for value in obj.values()] for obj in listData]
        cursor.execute(stat, values)
        conn.commit()
        count = cursor.rowcount
        print(count, "Record inserted successfully into jptrvl")
        cursor.close()

    except exc.SQLAlchemyError as e:
        print("Failed to insert record into table", e)
        conn.rollback()

    finally:
        if cursor:
            cursor.close()
            conn.close()


if __name__ == '__main__':
    getMainData()
    getDetailData()

[{'author': 'IkarusWillie',
  'content': '標題: [心得] 日本北海道 ラビスタ(LA VISTA)釧路川時間: Tue Dec 31 23:29:40 2019   旅 '
             '館  名 稱 ：ラビスタ釧路川 （LA VISTA 釧路川）   住 宿  時 間 ：2019年/2月   地        址 '
             '：北海道釧路市北大通2-1   電        話 ：0154-31-5489   入住/退房時間：15:00~ / '
             '11:00   早 餐  時 間 ：6:30~9:30   住 宿  房 型 '
             '：ダブルルーム（單人入住）、和風ツインルーム（雙人入住）   入 住  價 格 ：                    '
             'ダブルルーム    和風ツインルーム              2/20     9,390日圓          '
             '11,990日圓              2/21    11,090日圓          '
             '15,990日圓              2/22    10,390日圓          '
             '14,990日圓北海道振興優惠折價     3,000日圓          20,000日圓        折扣後總價    '
             '27,870日圓          22,970日圓   可 否  刷 卡 ：可   官        網 '
             '：https://www.hotespa.net/hotels/kushirogawa/圖文版請參閱：https://kimoriwillie.pixnet.net/blog/post/33047008從阿寒湖返回釧路後先到在釧路要住宿3個晚上的ラビスタ釧路川辦理入住及放行李我們全家訂了雙床的和洋室及單床的洋室各一間由於正好有搶到北海道振興優惠折價的名額算是還滿划算的不過當初訂房網站針對單人房的折價幅度比較少就是了旅館為我們安排同在12樓的兩間房間到了房間後發現兩間房